In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
file_name = 'title.basics.tsv'
imdb_titles = pd.read_csv(file_name, sep='\t', on_bad_lines='skip')
file_name = 'title.ratings.tsv'
imdb_ratings = pd.read_csv(file_name, sep='\t', on_bad_lines='skip')
file_name = 'movies.csv'
movieLens_movies = pd.read_csv(file_name)
file_name = 'ratings.csv'
movieLens_ratings = pd.read_csv(file_name)
file_name = 'links.csv'
movieLens_links = pd.read_csv(file_name)

<ipython-input-2-0e0d677ce438>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_titles = pd.read_csv(file_name, sep='\t', on_bad_lines='skip')


In [ ]:
print(imdb_ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496265 entries, 0 to 1496264
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1496265 non-null  object 
 1   averageRating  1496265 non-null  float64
 2   numVotes       1496265 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 34.2+ MB
None


In [ ]:
print(imdb_titles.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4533939 entries, 0 to 4533938
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 311.3+ MB
None


In [ ]:
print(movieLens_links.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB
None


In [ ]:
print(movieLens_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


In [ ]:
print(movieLens_ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


In [ ]:
print(movieLens_ratings.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
print(movieLens_links)
print(imdb_titles)

      movieId   imdbId    tmdbId
0           1   114709     862.0
1           2   113497    8844.0
2           3   113228   15602.0
3           4   114885   31357.0
4           5   113041   11862.0
...       ...      ...       ...
9737   193581  5476944  432131.0
9738   193583  5914996  445030.0
9739   193585  6397426  479308.0
9740   193587  8391976  483455.0
9741   193609   101726   37891.0

[9742 rows x 3 columns]
             tconst  titleType  \
0         tt0000001      short   
1         tt0000002      short   
2         tt0000003      short   
3         tt0000004      short   
4         tt0000005      short   
...             ...        ...   
4533934  tt16984776  tvEpisode   
4533935   tt1698478      short   
4533936  tt16984780  tvEpisode   
4533937  tt16984788  tvEpisode   
4533938   tt1698479      short   

                                              primaryTitle  \
0                                               Carmencita   
1                                   Le clown e

In [ ]:
# Convert 'imdbId' in movieLens_links to string to ensure compatibility
movieLens_links['imdbId'] = movieLens_links['imdbId'].astype(str)

# Remove the "tt" prefix and leading zeros from 'tconst' in imdb_titles
imdb_titles['tconst'] = imdb_titles['tconst'].str.replace('tt', '').astype(int).astype(str)

# Now perform the merge operations
# First merge links with movieLens_movies on 'movieId'
movies_with_imdbId = pd.merge(movieLens_movies, movieLens_links, on='movieId', how='inner')

# Merge with imdb_titles using the modified 'imdbId' and 'tconst'
merged_data = pd.merge(movies_with_imdbId, imdb_titles, left_on='imdbId', right_on='tconst', how='inner')

# Merge with imdb_ratings to get ratings
final_data = pd.merge(merged_data, imdb_ratings, on='tconst', how='left')

# Select and rename columns to match the ER diagram schema
final_movies = final_data.rename(columns={
    'movieId': 'movieId',
    'title_x': 'title',  # Title from movieLens_movies
    'genres_x': 'genre',  # Genre from movieLens_movies or imdb_titles
    'runtimeMinutes': 'duration',
    'startYear': 'releaseYear',
    'averageRating': 'rating'
})[['movieId', 'title', 'genre', 'duration', 'releaseYear', 'rating']]

# Display the final table structure
final_movies = final_movies.drop_duplicates(subset=['title'])

In [ ]:
movieLens_ratings['movieId'] = movieLens_ratings['movieId'].astype(str)


In [ ]:
print(movieLens_ratings)

        userId movieId  rating   timestamp
0            1       1     4.0   964982703
1            1       3     4.0   964981247
2            1       6     4.0   964982224
3            1      47     5.0   964983815
4            1      50     5.0   964982931
...        ...     ...     ...         ...
100831     610  166534     4.0  1493848402
100832     610  168248     5.0  1493850091
100833     610  168250     5.0  1494273047
100834     610  168252     5.0  1493846352
100835     610  170875     3.0  1493846415

[100836 rows x 4 columns]


In [ ]:
# Convert columns for compatibility
movieLens_links['imdbId'] = movieLens_links['imdbId'].astype(str)
imdb_ratings['tconst'] = imdb_ratings['tconst'].str.replace('tt', '').astype(int).astype(str)
movieLens_movies['movieId'] = movieLens_movies['movieId'].astype(str)
movieLens_ratings['movieId'] = movieLens_ratings['movieId'].astype(str)

# Merge movieLens_links with IMDb ratings on 'imdbId' and 'tconst'
merged_ratings = pd.merge(movieLens_links, imdb_ratings, left_on='imdbId', right_on='tconst', how='inner')

# Cap the number of votes to prevent memory crashes
VOTE_CAP = 100
merged_ratings['numVotesCapped'] = merged_ratings['numVotes'].clip(upper=VOTE_CAP)

# Expand IMDb ratings by capped numVotes
imdb_expanded = merged_ratings.loc[merged_ratings.index.repeat(merged_ratings['numVotesCapped'])].copy()
imdb_expanded['rating'] = imdb_expanded['averageRating']
imdb_expanded['userId'] = None  # IMDb has no userId
imdb_expanded['timestamp'] = None
imdb_expanded = imdb_expanded[['movieId', 'userId', 'rating', 'timestamp']]

# Adjust MovieLens ratings by multiplying by 2
movieLens_ratings['rating'] = movieLens_ratings['rating'] * 2
movieLens_adjusted = movieLens_ratings[['movieId', 'userId', 'rating', 'timestamp']]

print(movieLens_adjusted)

# Combine IMDb and MovieLens ratings
final_ratings = pd.concat([imdb_expanded, movieLens_adjusted], ignore_index=True)

# Display the structure of the final DataFrame
print(final_ratings.head())
print(final_ratings.info())


       movieId  userId  rating   timestamp
0            1       1     8.0   964982703
1            3       1     8.0   964981247
2            6       1     8.0   964982224
3           47       1    10.0   964983815
4           50       1    10.0   964982931
...        ...     ...     ...         ...
100831  166534     610     8.0  1493848402
100832  168248     610    10.0  1493850091
100833  168250     610    10.0  1494273047
100834  168252     610    10.0  1493846352
100835  170875     610     6.0  1493846415

[100836 rows x 4 columns]
  movieId userId  rating timestamp
0       1   None     8.3      None
1       1   None     8.3      None
2       1   None     8.3      None
3       1   None     8.3      None
4       1   None     8.3      None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072507 entries, 0 to 1072506
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   movieId    1072507 non-null  object 
 1   user

        userId movieId  rating   timestamp
0            1       1     4.0   964982703
1            1       3     4.0   964981247
2            1       6     4.0   964982224
3            1      47     5.0   964983815
4            1      50     5.0   964982931
...        ...     ...     ...         ...
100831     610  166534     4.0  1493848402
100832     610  168248     5.0  1493850091
100833     610  168250     5.0  1494273047
100834     610  168252     5.0  1493846352
100835     610  170875     3.0  1493846415

[100836 rows x 4 columns]


ValueError: You are trying to merge on int64 and object columns for key 'movieId'. If you wish to proceed you should use pd.concat

In [ ]:
final_movies
final_ratings

,movieId,userId,rating,timestamp
0,1,None,8.3,None
1,1,None,8.3,None
2,1,None,8.3,None
3,1,None,8.3,None
4,1,None,8.3,None
...,...,...,...,...
1072502,166534,610,8.0,1493848402
1072503,168248,610,10.0,1493850091
1072504,168250,610,10.0,1494273047
1072505,168252,610,10.0,1493846352


In [ ]:
# prompt: drop null values for these tables

# Drop rows with any null values in all DataFrames
final_movies.dropna(inplace=True)
final_ratings.dropna(inplace=True)


In [ ]:
final_movies.to_csv('final_movies.csv', index=False)
final_ratings.to_csv('final_ratings_new.csv', index=False)


In [ ]:
final_ratings[final_ratings['timestamp'].isnull()]

,movieId,userId,rating,timestamp


In [ ]:
# prompt: can you find all ratings from the final_ratings that are above 10

# Assuming final_ratings DataFrame is already created as in the provided code.

ratings_above_10 = final_ratings[final_ratings['rating'] > 10]
ratings_above_10

,movieId,userId,rating,timestamp
